In [2]:
### HEATMAPS
#import dependencies
import gmaps
import pandas as pd

# Google developer API key
from rb_gkey import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [3]:
# Create dfs for crime data
df_latlong = pd.read_csv('../harmonious_hummingbirds/Intoxication_Data.csv')
df_duis = df_latlong
df_latlong.dropna()
df_latlong.head()

FileNotFoundError: [Errno 2] File b'../harmonious_hummingbirds/Intoxication_Data.csv' does not exist: b'../harmonious_hummingbirds/Intoxication_Data.csv'

In [232]:
#Drop columns we don't need
df_latlong = df_latlong.drop(['Time','Area ID','Charge','Area ID','Area Name','Descent Code','Charge Group Code','Reporting District'], axis=1)

In [234]:
#Check labels
df_latlong.nunique()

Report ID                   280105
Arrest Date                   3684
Age                             84
Sex Code                         2
Charge Group Description         8
Arrest Type Code                 3
Charge Description              63
Address                      23856
Cross Street                 10836
Location                     27648
dtype: int64

In [235]:
#rename column without spaces
df_latlong = df_latlong.rename(columns={"Charge Group Description": "CGD"})

In [236]:
#filter for alcohol
df_latlong = df_latlong[~df_latlong.CGD.isin(["Gambling", "Narcotic Drug Laws", 'Driving Under Influence','Miscellaneous Other Violations','Homicide','Moving Traffic Violations'])]

In [237]:
#check
df_latlong["Charge Description"].unique()

array(['OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE', 'DRINKING IN PUBLIC',
       'DRINKING ALCOHOL*******',
       'SALE/ETC OF ALCOHOLIC BEVERAGE TO MINOR', 'DRINKING IN PUBLIC***',
       'MINOR POSSESS ALCOHOL', 'DISORD CONDUCT:UI/DRUG, ALCOHOL',
       'DRINKING ALCOHOL', 'SELL/LET MINOR CONSUME ALCOHOL',
       'ALCOHOLIC BEV LICENSEE POSS UNAUTH LIQ',
       'MINOR IN POSSESSION OF ALCOHOLIC BEVERAGE',
       'ALCOHOLIC BEV SALES TO INTOXICATED PERSNS',
       'PERMIT BEGGING/SOLICITING OF ALCOHOL',
       'CONSUME/PURCHASE ALCOHOLIC BEV BY MINOR',
       'TRANSPORT ALCOHOLIC BEVERAGE BY MINOR',
       'SELL/PURCHASE ALCOHOL AFTER HOURS', 'DRUNK, ALCOHOL',
       'ALCOHOLIC BEV LIC PERMIT ALCHL CONSMPTN',
       'OFFER OF SUBSTITUTE ALCOHOLIC BEV BRAND',
       'POSSESS ANY ALCOHOLIC BEV LIST IN BP2535',
       'MINOR IN POSS OF ALCOHOL BEVERAGE IN VEH',
       'MFG/SELL/ETC ALCOHOL CONTAIN POISN SUBS',
       'MINOR ATT PURCHASE ALCOHOLIC BEVERAGE',
       'EMPLOY MINOR WHERE ALCO

In [240]:
df_latlong = df_latlong[~df_latlong['Charge Description'].isin([
    'SALE/ETC OF ALCOHOLIC BEVERAGE TO MINOR', 'DISORD CONDUCT:UI/DRUG, ALCOHOL', 
    'SELL/LET MINOR CONSUME ALCOHOL','ALCOHOLIC BEV LICENSEE POSS UNAUTH LIQ',
    'ALCOHOLIC BEV SALES TO INTOXICATED PERSNS', 'PERMIT BEGGING/SOLICITING OF ALCOHOL',
    'SELL/PURCHASE ALCOHOL AFTER HOURS','ALCOHOLIC BEV LIC PERMIT ALCHL CONSMPTN',
    'OFFER OF SUBSTITUTE ALCOHOLIC BEV BRAND', 'POSSESS ANY ALCOHOLIC BEV LIST IN BP2535',
    'MFG/SELL/ETC ALCOHOL CONTAIN POISN SUBS', 'MINOR ATT PURCHASE ALCOHOLIC BEVERAGE',
    'EMPLOY MINOR WHERE ALCOHOL SOLD','MINOR EMPLOYED WHERE ALCOHOL IS SOLD','POSS OF OPEN CONTAINER OF ALCOHOL IN PARK',
    'FURNISH ALCOHOLIC BEVERAGE TO A MINOR'])]

In [241]:
#check
df_latlong["Charge Description"].unique()

array(['OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE', 'DRINKING IN PUBLIC',
       'DRINKING ALCOHOL*******', 'DRINKING IN PUBLIC***',
       'MINOR POSSESS ALCOHOL', 'DRINKING ALCOHOL',
       'MINOR IN POSSESSION OF ALCOHOLIC BEVERAGE',
       'CONSUME/PURCHASE ALCOHOLIC BEV BY MINOR',
       'TRANSPORT ALCOHOLIC BEVERAGE BY MINOR', 'DRUNK, ALCOHOL',
       'MINOR IN POSS OF ALCOHOL BEVERAGE IN VEH'], dtype=object)

In [242]:
#check
df_latlong["Charge Description"].value_counts()

DRINKING IN PUBLIC                           93978
OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE      26125
DRINKING ALCOHOL                             10190
DRINKING IN PUBLIC***                         9383
MINOR IN POSSESSION OF ALCOHOLIC BEVERAGE     4695
DRINKING ALCOHOL*******                       2642
MINOR POSSESS ALCOHOL                         1218
CONSUME/PURCHASE ALCOHOLIC BEV BY MINOR        180
DRUNK, ALCOHOL                                 113
MINOR IN POSS OF ALCOHOL BEVERAGE IN VEH        37
TRANSPORT ALCOHOLIC BEVERAGE BY MINOR           16
Name: Charge Description, dtype: int64

In [243]:
#extract coordinates from current formula
df_coordinates = df_latlong.Location.str.extract(
    '^(?P<Location>.*)\s*\((?P<Latitude>[^,]*),\s*(?P<Longitude>\S*)\).*$',
    expand=True
)

In [244]:
# Merge two Dataframes on index of both the dataframes
df_merged = df_latlong.merge(df_coordinates, left_index=True, right_index=True)
df_merged

,Report ID,Arrest Date,Age,Sex Code,CGD,Arrest Type Code,Charge Description,Address,Cross Street,Location_x,Location_y,Latitude,Longitude
0,190325212,2019-11-23,21,F,Liquor Laws,I,OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE,28TH ST,E UNIVERSITY,"(34.0278, -118.281)",,34.0278,-118.281
2,190818921,2019-11-27,42,M,Drunkeness,M,DRINKING IN PUBLIC,SEPULVEDA BL,PEARL,"(34.0344, -118.4336)",,34.0344,-118.4336
5,190112235,2019-04-26,57,M,Drunkeness,M,DRINKING IN PUBLIC,6TH,HILL,"(34.0481, -118.2542)",,34.0481,-118.2542
6,190112276,2019-04-27,35,M,Liquor Laws,I,DRINKING ALCOHOL*******,7TH,FLOWER,"(34.0438, -118.2547)",,34.0438,-118.2547
7,190210388,2019-04-22,20,M,Liquor Laws,I,OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE,2200 W 6TH ST,NaN,"(34.0601, -118.2761)",,34.0601,-118.2761
...,...,...,...,...,...,...,...,...,...,...,...,...,...
277282,5507464,2018-12-28,28,F,Drunkeness,M,"DRUNK, ALCOHOL",1800 S WESTMORELAND AV,NaN,"(34.0416, -118.2896)",,34.0416,-118.2896
277314,5503105,2018-12-22,19,M,Liquor Laws,M,MINOR POSSESS ALCOHOL,10900 WILMINGTON AV,NaN,"(33.9363, -118.2426)",,33.9363,-118.2426
277972,5530677,2019-01-26,36,M,Liquor Laws,M,OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE,14600 CALVERT ST,NaN,"(34.1817, -118.4509)",,34.1817,-118.4509
279740,5597027,2019-04-10,20,M,Liquor Laws,M,MINOR IN POSSESSION OF ALCOHOLIC BEVERAGE,COVELLO,ZOMBAR,"(34.2076, -118.4673)",,34.2076,-118.4673


In [245]:
#change data to float
df_merged

crime_locations = df_merged[["Latitude", "Longitude"]]
crime_locations.dtypes

crime_locations["Latitude"] = crime_locations.Latitude.astype(float)
crime_locations["Longitude"] = crime_locations.Longitude.astype(float)

crime_locations

/Users/andrewwright/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/andrewwright/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,Latitude,Longitude
0,34.0278,-118.2810
2,34.0344,-118.4336
5,34.0481,-118.2542
6,34.0438,-118.2547
7,34.0601,-118.2761
...,...,...
277282,34.0416,-118.2896
277314,33.9363,-118.2426
277972,34.1817,-118.4509
279740,34.2076,-118.4673


In [246]:
#sample data
df_sampler = crime_locations.sample(100000)

In [247]:
#import bars
df_bars = pd.read_csv('Output Files/bars_and_crime.csv')
df_bars

,name,review_count,lat,long,price,count of crimes,count_of_crimes
0,The Penthouse at Mastro's,4650,34.068933,-118.398747,$$$$,53,53
1,Gracias Madre - West Hollywood,4101,34.080795,-118.386972,$$,73,73
2,Taste on Melrose,2627,34.082573,-118.374798,$$,132,132
3,GRANVILLE,2622,34.145109,-118.255093,$$,40,40
4,Fig & Olive - Melrose Place,2488,34.083475,-118.376174,$$$,124,124
5,PUMP Restaurant,2408,34.083713,-118.385255,$$$,78,78
6,Firefly,2226,34.141468,-118.388418,$$$,379,379
7,The Abbey Food & Bar,2192,34.083610,-118.385375,$$,77,77
8,Kitchen 24,2146,34.100373,-118.329342,$$,919,919
9,Clifton's Republic,2011,34.045400,-118.252940,$$,3432,3432


In [248]:
bar_locations = df_bars[["lat", "long"]]
bar_locations.dtypes

#Check for errors
bar_locations


,lat,long
0,34.068933,-118.398747
1,34.080795,-118.386972
2,34.082573,-118.374798
3,34.145109,-118.255093
4,34.083475,-118.376174
5,34.083713,-118.385255
6,34.141468,-118.388418
7,34.083610,-118.385375
8,34.100373,-118.329342
9,34.045400,-118.252940


In [249]:
# Plot Heatmap
fig = gmaps.figure()

#marker top 50 bars
markers = gmaps.marker_layer(bar_locations)
fig.add_layer(markers)

# Create heat layer
heat_layer = gmaps.heatmap_layer(df_sampler, dissipating=False, max_intensity=50, point_radius=0.003)
 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [250]:
#Output to CSV
df_merged.to_csv("Output Files/df_crime.csv", index=False, header= True)

In [251]:
# DUI Map DF
df_duis

,Report ID,Arrest Date,Time,Area ID,Area Name,Reporting District,Age,Sex Code,Descent Code,Charge Group Code,Charge Group Description,Arrest Type Code,Charge,Charge Description,Address,Cross Street,Location
0,190325212,2019-11-23,1315.0,3,Southwest,328,21,F,W,17.0,Liquor Laws,I,25620BP,OPEN ALCOHOLIC BEV IN PUBLIC PARK/PLACE,28TH ST,E UNIVERSITY,"(34.0278, -118.281)"
1,5809994,2019-11-27,2100.0,18,Southeast,1801,20,M,H,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,FIGUEROA,MANCHESTER,"(33.9601, -118.2827)"
2,190818921,2019-11-27,1645.0,8,West LA,893,42,M,W,18.0,Drunkeness,M,41.27CLAMC,DRINKING IN PUBLIC,SEPULVEDA BL,PEARL,"(34.0344, -118.4336)"
3,5672540,2019-11-24,356.0,15,N Hollywood,1547,30,M,H,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,BURBANK,HOLLYWOOD FY,"(34.1673, -118.3816)"
4,5806232,2019-11-23,149.0,2,Rampart,239,25,M,H,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,WESTERN AV,101 SOUTHBOUND,"(34.0609, -118.2535)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280100,5606768,2019-04-21,245.0,7,Wilshire,759,28,M,O,22.0,Driving Under Influence,F,23153(F)VC,DUI OF A DRUG CAUSING INJURY,PICO BL,5TH AV,"(34.0475, -118.323)"
280101,5606554,2019-04-20,2100.0,15,N Hollywood,1521,27,M,H,22.0,Driving Under Influence,M,23152(F)VC,DUI OF ANY DRUG,SHERMAN WY,ETHEL AV,"(34.2012, -118.4211)"
280102,5610860,2019-04-25,1925.0,9,Van Nuys,935,48,M,H,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,6200 SYLMAR AV,NaN,"(34.1827, -118.4465)"
280103,9391083,2010-01-19,35.0,8,West LA,842,29,F,H,22.0,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,WILSHIRE BL,GRANVILLE AV,"(34.048, -118.4626)"


In [252]:
#Filter for DUI charges

df_coordinates_dui = df_duis.Location.str.extract(
    '^(?P<Location>.*)\s*\((?P<Latitude>[^,]*),\s*(?P<Longitude>\S*)\).*$',
    expand=True)

df_duis
# Merge two Dataframes on index of both the dataframes
df_merger = df_duis.merge(df_coordinates_dui, left_index=True, right_index=True)
df_merger

df_merger

crime_locations_dui = df_merger[["Latitude", "Longitude"]]
crime_locations_dui.dtypes

crime_locations_dui["Latitude"] = crime_locations_dui.Latitude.astype(float)
crime_locations_dui["Longitude"] = crime_locations_dui.Longitude.astype(float)

crime_locations_dui

/Users/andrewwright/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/andrewwright/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Latitude,Longitude
0,34.0278,-118.2810
1,33.9601,-118.2827
2,34.0344,-118.4336
3,34.1673,-118.3816
4,34.0609,-118.2535
...,...,...
280100,34.0475,-118.3230
280101,34.2012,-118.4211
280102,34.1827,-118.4465
280103,34.0480,-118.4626


In [253]:
#Sample to decrease Load
df_dui_sampler = crime_locations_dui.sample(10000)

In [254]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(df_dui_sampler, dissipating=False, max_intensity=30, point_radius=0.008
)
 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))